## Подготовка

In [388]:
# Предварительная настройка системы
import numpy as np
import pandas as pd

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)

import datetime
from datetime import datetime, date

In [389]:
# Каталог с CSV-файлами
src_csv_dir = '../csv/'
src_data_file = 'data_for_spb_hakaton_entities1-Table 1.csv'  # данные хакатона
src_history_file  = 'history-Table 1.csv'                         # история изменений
src_sprints_file  = 'sprints-Table 1.csv'                         # спринты

### Спринты

In [391]:
# Загружаем исходник спринтов
dS = pd.read_csv(f"{src_csv_dir}{src_sprints_file}",
                    header=1,
                    parse_dates=[
                        'sprint_start_date',
                        'sprint_end_date'
                    ],
                    delimiter=";"
                  )
# dS.head(10)

In [392]:
# Считываем данные для таблицы связей спринтов и задач
dSE_add_list = list([])
for _, row in dS[['sprint_name', 'entity_ids']].iterrows():
    esl = list(map(int, row['entity_ids'].strip("{}").split(",")))
    for l in esl:
        dSE_add_list.append({
            'entity_id': l,
            'sprint': row['sprint_name']  # TODO Заменить на ID
        })

# dSE_add_list[:20]

In [393]:
# Создаём таблицу связей спринтов и задач
dSE = pd.DataFrame(dSE_add_list)
dSE['entity_id'] = pd.to_numeric(dSE['entity_id'])

# dSE.dtypes

### История

In [395]:
# Загружаем историю
dH = pd.read_csv(f"{src_csv_dir}{src_history_file}",
                     header=1,
                     delimiter=";"
                )

# dH

In [396]:
# Удаляем лишние колонки в истории
dH.drop(columns=['Столбец1', 'Unnamed: 7'], inplace=True)
# dH.dtypes
dH

       entity_id   history_property_name   history_date  history_version  \
0        94297.0  Время решения 3ЛП ФАКТ  9/10/24 11:17              1.0   
1        94297.0    Время решения (ФАКТ)  9/10/24 11:17              1.0   
2        94297.0             Исполнитель  7/13/23 11:07              1.0   
3            NaN                     NaN            NaN              NaN   
4        94297.0             Исполнитель  7/21/23 11:06              3.0   
...          ...                     ...            ...              ...   
64174  5179881.0                  Спринт  10/23/24 7:00             11.0   
64175  5179881.0                  Спринт  10/23/24 7:00             11.0   
64176  5179881.0                  Спринт  10/23/24 7:00             11.0   
64177  5179881.0               Резолюция  10/23/24 7:00             12.0   
64178  5179881.0                  Статус  10/23/24 7:00             12.0   

      history_change_type                                     history_change  
0       

In [397]:
# Нормализуем типы данных в истории
dH['entity_id'] = dH['entity_id'].astype('Int64')
dH['history_version'] = dH['history_version'].astype('Int64')
dH['history_date'] = pd.to_datetime(dH['history_date'], format='%m/%d/%y %H:%M', errors='coerce')
# dH.dtypes
# dH

In [398]:
# Готовим данные для таблицы истории изменений статусов
dHS_pre = dH.loc[dH['history_property_name'] == 'Статус']
dHS_add_list = list([])
for index, row in dHS_pre.iterrows():
    (before, after) = row['history_change'].split(" -> ")
    dHS_add_list.append({
        'entity_id': row['entity_id'],
        'history_date': row['history_date'],
        'history_version': row['history_version'],
        'before': before,
        'after': after
    })

# dHS_add_list[:20]

In [399]:
# Заполняем таблицу истории изменений статусов
dHS = pd.DataFrame(dHS_add_list)
dHS['entity_id'] = dHS['entity_id'].astype('Int64')
dHS['history_version'] = dHS['history_version'].astype('Int64')
dHS['history_date'] = pd.to_datetime(dHS['history_date'])
# dHS

### Данные

In [401]:
dD = pd.read_csv(f"{src_csv_dir}{src_data_file}",
                    header=1,               # в первой строчке мусор, во второй — названия колонок
                    index_col='entity_id',  # ключ в первой колонке
                    dtype = {
                                'parent_ticket_id': 'Int64',
                                'estimation': 'Int64',
                                'spent': 'Int64'
                            },              # есть незаполненные parent_ticket_id, поэтому Int64, не int
                    parse_dates = [
                                    'create_date',
                                    'update_date'
                                  ],        # в этих колонках легко распознаваемые даты (и всегда есть)
                    delimiter=';')          # нетипичный csv

# dD.dtypes

In [402]:
# Справочник названий команд
# TODO Лучше сделать через dataframe
area_ref = {
    "Система. Движок": "Движок",
    "Система.Вики": "Wiki",
    "Система.Ошибки": "Ошибки",
    "Система.Таск-трекер": "Task tracker",
    "Система.ХранениеАртефактов": "Хранение",
    "Управление релизами изменениями": "Релизы",
    }
dD['area'] = dD['area'].replace(area_ref)
# dD.groupby('area').size()

In [403]:
# Справочник статусов задач
status_ref = {
'В ожидании': 'waiting',
'Готово к разработке': 'readyForDevelopment',
'СТ Завершено': 'stCompleted',
'Локализация': 'localization',
'Подтверждение': 'verification',
'СТ': 'st',
'Разработка': 'development',
'Отложен': 'hold',
'Исправление': 'fixing',
'Анализ': 'analysis',
'Подтверждение исправления': 'verification',
'Тестирование': 'testing',
'Отклонен исполнителем': 'rejectedByThePerformer',
'В работе': 'inProgress',
'Создано': 'created',
'Выполнено': 'done',
'Закрыто': 'closed'
}
dD['status'] = dD['status'].replace(status_ref)

dD['status']

entity_id
94297                      closed
102481                     closed
1805925                   testing
1934905                    closed
1943849    rejectedByThePerformer
                    ...          
5179477                    closed
5179714                    closed
5179793                    closed
5179847                    closed
5179881                    closed
Name: status, Length: 2485, dtype: object

In [404]:
# Удалить лишние колонки
dD.drop(columns=['created_by', 'updated_by', 'assignee', 'owner',
                 'parent_ticket_id', 'state', 'due_date', 'rank',
                 'workgroup', 'resolution'], inplace=True)
# dD.dtypes

In [405]:
# Почистить названия задач
dD['name'] = dD['name'].apply(lambda x: x.strip(" \t"))
# dD['name']
# TODO Что-то не так с задачей 5179847

## Сохранение

In [407]:
# Настройки сохранения
dir_pickle = '../pickle/'
data_pickle = 'data.pkl'
history_pickle = 'history.pkl'
history_statuses_piclke = 'history_statuses.pkl'
sprints_pickle = 'sprints.pkl'
sprints_tasks_pickle = 'sprints_tasks.pkl'

In [409]:
# Сохраняем
dD.to_pickle(f"{dir_pickle}{data_pickle}")
dS.to_pickle(f"{dir_pickle}{sprints_pickle}")
dSE.to_pickle(f"{dir_pickle}{sprints_tasks_pickle}")
dH.to_pickle(f"{dir_pickle}{history_pickle}")
dHS.to_pickle(f"{dir_pickle}{history_statuses_piclke}")
# TODO Неаккуратно настроено, переписать